In [2]:
from scipy.io import loadmat
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

data1 = loadmat('lab2_1.mat')
data2 = loadmat('lab2_2.mat')
data3 = loadmat('lab2_3.mat')

al = data2['al']
bl = data2['bl']
cl = data2['cl']

In [18]:
mu_hat = np.sum(al, axis=0) / al.size
sigma_hat = np.sum((al - mu_hat)@(al - mu_hat).T) / al.size
sigma_hat

1721668.8199999996